<a href="https://colab.research.google.com/github/s528661/nlp100/blob/main/nlp100_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 80. ID番号への変換

問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [7]:
!pip install mecab-python3

In [8]:
import re,sys,os

import pickle
import time
import copy

import math
import random
import string

from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import scipy as scp

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn import feature_extraction, preprocessing

from sklearn import svm
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, f1_score
from sklearn.metrics import accuracy_score,  precision_score, recall_score, confusion_matrix, roc_auc_score

# import dask.array as da

import lightgbm as lgb

# Neural Language Processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest

from sklearn.linear_model import RidgeClassifier
import MeCab

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [9]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip --no-check-certificate
!unzip ./NewsAggregatorDataset.zip

--2023-12-09 03:40:08--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas     [  <=>               ]  27.87M  91.9MB/s    in 0.3s    

2023-12-09 03:40:09 (91.9 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203]

Archive:  ./NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [10]:
data = pd.read_csv('newsCorpora.csv',delimiter='\t',header=None,
                 names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])

In [11]:
fix_data = data[data['PUBLISHER'].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]

In [12]:
def preprocessing(text):
  text = "".join([i for i in text if i not in string.punctuation])
  text = text.lower()
  text = re.sub("[0-9]+", "", text)
  return text

In [13]:
fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))

<ipython-input-13-0330034268ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))


In [14]:
train_all, test = train_test_split(fix_data, test_size=0.1, shuffle=True)

In [15]:
train, val = train_test_split(train_all, test_size=0.1, shuffle=True)

In [16]:
train.to_csv("train.txt", sep="\t", index=False, header=None)
val.to_csv("valid.txt", sep="\t", index=False, header=None)
test.to_csv("test.txt", sep="\t", index=False, header=None)

In [17]:
def preprocessing(text):
  text = "".join([i for i in text if i not in string.punctuation])
  text = text.lower()
  text = re.sub("[0-9]+", "", text)
  return text

In [18]:
fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))

<ipython-input-18-0330034268ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))


In [19]:
#vectorizer = TfidfVectorizer(min_df=10, ngram_range=(1, 2)) # 1-gram, 2-gramでTfidfを計算
#X = vectorizer.fit_transform(fix_data['pTITLE']).toarray()
#X = pd.DataFrame(X, columns=vectorizer.get_feature_names_out())
X = fix_data['pTITLE']

In [20]:
X_train_all, X_test = train_test_split(X, test_size=0.1, shuffle=True)

In [21]:
X_train, X_val = train_test_split(X_train_all, test_size=0.1, shuffle=True)

In [22]:
X_train

63748     hall of fame quarterback jim kelly avoids surg...
207090    pregnant women at higher risk for serious car ...
67831     selena gomez scores favourite female singer at...
154317    dont snort it and other powdered alcohol pro tips
46597     update consumers seek to raise california cap ...
                                ...                        
141671    michelle obama to guest star on nashville alon...
289604    wti brent pare gains as iraq unrest seen spari...
124588    yahoo after alibaba still set to get value fro...
288846    update jump street guns down dragon to lead we...
280125      chris christie busts a move on the tonight show
Name: pTITLE, Length: 10805, dtype: object

In [23]:
X_val

146169    philips quarterly profit misses estimates amid...
52749     katheryn deprill reunited with birth mother wh...
272015    european shares slip as euro zone recovery slo...
106719    dean mcdermott admits to cheating on tori spel...
361964    khloe kardashian and boyfriend french montana ...
                                ...                        
337639    rolling stones  rolling stones stars appear in...
71220                gm failed to fix defect twice congress
280522    justin bieber will plead guilty to reckless dr...
349947    us sues tmobile usa alleges bogus charges on p...
153861    amazon prime gets exclusive deals on classic h...
Name: pTITLE, Length: 1201, dtype: object

In [24]:
X_test

120010    owen and emmett ezell onceconjoined twins set ...
14516     mandy moore steps out with fresh look just hou...
292491    barack obama  president obamas daughter joins ...
61677     microsoft takes it back promises not to snoop ...
345146    gm owners buy new models while recalled autos ...
                                ...                        
261543    is elle fanning hollywoods coolest teen malefi...
188246    yellen avoids specifying a timetable for inter...
279464     jump street continues the jonah hill channing...
207458    midterm inflation projections key to ecb actio...
59118     christine mcvie to hit the road with fleetwood...
Name: pTITLE, Length: 1334, dtype: object

In [25]:
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [26]:
words = {}
for title in X_train:
  for word in title.rstrip().split():
    words[word] = 0

In [27]:
for title in X_train:
  for word in title.rstrip().split():
    words[word] += 1

In [28]:
id_words = [(idx, item) for idx, item in enumerate(words.items()) if item[1] >= 2]

In [29]:
id_words = sorted(id_words, key=lambda x: x[1][1], reverse=True)

In [30]:
id_words

[(44, ('to', 2916)),
 (61, ('in', 1938)),
 (28, ('the', 1630)),
 (1, ('of', 1476)),
 (17, ('for', 1355)),
 (48, ('on', 1344)),
 (8, ('as', 1165)),
 (41, ('update', 1027)),
 (51, ('us', 996)),
 (35, ('and', 975)),
 (65, ('a', 837)),
 (143, ('with', 770)),
 (14, ('at', 683)),
 (92, ('is', 610)),
 (67, ('after', 601)),
 (96, ('new', 542)),
 (666, ('says', 396)),
 (223, ('from', 383)),
 (121, ('up', 369)),
 (188, ('by', 327)),
 (229, ('kardashian', 276)),
 (660, ('over', 258)),
 (1138, ('her', 258)),
 (228, ('kim', 248)),
 (153, ('be', 238)),
 (161, ('first', 219)),
 (357, ('stocks', 218)),
 (318, ('china', 210)),
 (872, ('more', 208)),
 (103, ('will', 205)),
 (131, ('shares', 205)),
 (1876, ('about', 203)),
 (806, ('may', 201)),
 (320, ('are', 200)),
 (359, ('data', 198)),
 (780, ('not', 186)),
 (1043, ('out', 185)),
 (275, ('that', 179)),
 (569, ('euro', 179)),
 (34, ('it', 177)),
 (432, ('sales', 174)),
 (127, ('day', 169)),
 (258, ('fed', 168)),
 (331, ('you', 168)),
 (614, ('deal', 16

# 81. RNNによる予測  

In [31]:
dw = 300
dh = 50
class RNN(nn.Module):
  def __init__(self,dw,dh,n_word,n_label):
    super().__init__()
    self.rnn = nn.RNN(dw,dh,nonlinearity="tanh")
    self.emb = nn.Embedding(n_word,dw)
    self.layer = nn.Linear(dh,n_label,bias=True)
    self.softmax = nn.Softmax()

  def forward(self,x,h):
    x = self.emb(x)
    x, h = self.rnn(x,h=None)
    x = x[:,-1,:]
    x = self.softmax(x)
    return x

# 82. 確率的勾配降下法による学習  
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題81で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．